In [22]:
from pathlib import Path
import os
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
np.seterr(divide = 'ignore')

path = Path('./../data/')

def sample(softmax, temperature):

    EPSILON = 10e-16 # to avoid taking the log of zero
    softmax = (np.array(softmax) + EPSILON).astype('float64')
    
    preds = np.log(softmax) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)

    return probas[0]

def remove(preds, max_drop, temperature):
    
    to_drop = len(preds)*[0]
    for _ in range(max_drop):
        to_drop += (sample(preds, temperature))
        
    return to_drop

def plots(score, temperature):
    
    labels = [s[0] for s in score]
    preds = [s[1] for s in score]

    ks = [stats.ks_2samp(l, p) for l,p in zip(labels, preds)]
    auc = [metrics.roc_auc_score(l, p) for l,p in zip(labels, preds)]

    plt.plot(auc)
    plt.plot(len(auc)*[0.5])
    plt.xlabel("number of iterations")
    plt.ylabel('auc')
    plt.grid()
    plt.title(temperature)
    plt.show()
    
    plt.plot([k[0] for k in ks])
    plt.ylabel('ks-stats')
    plt.xlabel('number of iterations')
    plt.grid()
    plt.show()
    
    plt.plot([k[1] for k in ks], linewidth=2.0)
    plt.ylabel('p-value')
    plt.xlabel('number of iterations')
    plt.grid()
    plt.show()


## Attribute Comparison

Each row is labelled with its (survey) *source*.

- *Source*
- Geschlecht
- Alter
- Schulabschluss 
- Berufsgruppe
- Erwerbstätigkeit
- wöchentliche Arbeitszeit
- Berufsgruppe 
- BRS1, BRS2, BRS3, BRS4, BRS5, BRS6
- Optimismus
- Pessimismus 
- ASKU1, ASKU2, ASKU3
- IE_int1
- IE_int2 
- IE_ext1
- IE_ext2

### Preprocessing: Allensbach

In [5]:
allensbach_cols = ['S01', 'S02', 'S03_rec', 'S04_rec', 'S05',
                   'V04_01', 'V04_02', 'V04_03', 'V04_04', 'V04_05', 'V04_06',
                   'S06b_rec', 'V01', 'V22',
                   'V20_5', 'V20_6', 'V20_7',
                   'V20_1', 'V20_2', 'V20_3', 'V20_4']

names = ['Geschlecht', 'Alter', 'Schulabschluss', 'Berufsgruppe', 'Erwerbstätigkeit',
         'BRS1', 'BRS2', 'BRS3', 'BRS4', 'BRS5', 'BRS6',
         'woechentlicheArbeitszeit', 'Optimismus', 'Pessimismus',
         'ASKU1', 'ASKU2', 'ASKU3',
         'IE_int1', 'IE_int2', 'IE_ext1', 'IE_ext2']

allensbach = pd.read_csv(os.path.join(path, 'Allensbach/abs_data.csv'),
                         usecols = allensbach_cols,
                         delimiter=';', low_memory=False)

allensbach.replace(',', '.', regex=True, inplace=True)
allensbach.replace(' ', np.nan, regex=True, inplace=True)

allensbach.rename(columns= dict(zip(allensbach_cols, names)), inplace=True)
allensbach.Geschlecht = allensbach.Geschlecht -1

allensbach.head()

,Geschlecht,Alter,Erwerbstätigkeit,Optimismus,BRS1,BRS2,BRS3,BRS4,BRS5,BRS6,...,IE_int2,IE_ext1,IE_ext2,ASKU1,ASKU2,ASKU3,Pessimismus,Schulabschluss,Berufsgruppe,woechentlicheArbeitszeit
0,1,73,2,7,5,4,1,4,4,5,...,1,5,4,1,1,1,1,2,8,NaN
1,1,24,NaN,5,2,4,2,5,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,51,1,2,2,4,NaN,5,1,5,...,2,NaN,4,2,2,3,6,3,3,4
3,0,57,1,6,5,3,5,3,5,3,...,2,4,5,1,2,1,2,5,2,3
4,0,64,2,4,4,4,4,4,4,4,...,3,2,2,5,1,1,4,2,1,NaN


### Preprocessing: GBS

In [6]:
gbs1 = pd.read_csv(os.path.join(path, 'data/GBS/gbs1.csv'))
gbs2 = pd.read_csv(os.path.join(path, 'data/GBS/gbs2.csv'))

mapping1 = pd.read_csv(os.path.join(path, 'data/GBS/gbs_map1.csv'),
                                    encoding = "ISO-8859-1", delimiter = ';')
mapping2 = pd.read_csv(os.path.join(path, 'data/GBS/gbs_map2.csv'),
                                    encoding = "ISO-8859-1", delimiter = ';')

# Rename 'GBS-CODE' column
key = 'GBS-CODE'
mapping1 = mapping1.rename(columns={'GBS Code': key})
mapping2 = mapping2.rename(columns={'GBS Code': key})
gbs1 = gbs1.rename(columns={'gbs_code': key})
gbs2 = gbs2.rename(columns={'GBS-Code': key})

gbs__ = pd.read_csv(os.path.join(path, 'data/GBS/gbs1_p.csv'), sep=" ")
gbs__.columns = ['col']

personen = []

for i in range(gbs__.shape[0]):
    person_data = str(gbs__.loc[i].values)
    if ('mehrere Personen' in person_data):
        personen.append(int (person_data.split("mehrere Personen, und zwar:\",",1)[1][0]))
    else:
        personen.append(1)
        
gbs1['am24'] = personen


# Merge GBS "Umfrage Wellen" and create dataframe
gbs = pd.merge(gbs1, gbs2, how='inner', on=key)
gbs = pd.merge(gbs, mapping1, how='left', on=key)
gbs = pd.merge(gbs, mapping2, how='left', on=key)


gbs_cols = [ 'am01', 'am02_02', 'am14', 'am22', 'am17',
           'br01_01', 'br01_02', 'br01_03', 'br01_04', 'br01_05', 'br01_06',
           'am20_01', 'so01_01', 'so02_01',
           'sw02_01', 'sw02_02', 'sw02_03',
           'ie01_01', 'ie01_02', 'ie01_03', 'ie01_04']

#add gesis columns
gesis_attributes = ['am24', 'am02_01', 'am11','am10','am23','am14','am15','am29','am27',
                    'gh03_08','gh08_03','gh05_01','pt04', 'pt03_01','pt01_02','gh07_01','gh04_01',
                    'gh03_10','gh03_07','gh03_13','gh03_11', 'v2673_1_bfi_bfi1','v2673_1_bfi_bfi2',
                    'v2673_1_bfi_bfi3',
                    'v2673_1_bfi_bfi4','v2673_1_bfi_bfi5','v2673_1_bfi_bfi6','v2673_1_bfi_bfi7',
                    'v2673_1_bfi_bfi8',
                    'v2673_1_bfi_bfi9','v2673_1_bfi_bfi10', 'gh03_09','lo01_04','pt06_01','br01_06']

gesis_name = ['Personen im Haushalt','Geburtsjahr', 'Geburtsland','Nationalitaet','Familienstand',
              'Hoechster Bildungsabschluss','Berufliche Ausbildung',
            'Nettoeinkommen Selbst','Nettoeinkommen Haushalt', 'Schlechter Schlaf',
              'Leben genießen','Alles anstrengend','Wahlteilnahme','Wahlabsicht',
              'Desinteresse Politiker','Zufriedenheit Leben','Aktiv','Verärgert','Wach','Nervös','Ängstlich',
              'Zurueckhaltend','leicht Vertrauen','Faulheit','Entspannt','wenig kuenstlerisches Interesse',
              'Gesellig',
              'Andere kritisieren','Gruendlich','Nervoes','Phantasievoll','Druck','Optimismus Zukunft',
              'Zufriedenheit Wahlergebnis','Resilienz']

gbs = gbs[[key] + ['Gruppe'] + gbs_cols + gesis_attributes]
gbs.columns = [key] + ['Gruppe']  + names + gesis_name

del mapping1, mapping2, gbs1, gbs2

gbs = gbs.sort_values(by=['GBS-CODE'])


In [7]:
def merge(arr1, arr2):
    result = []
    for i,j in zip(arr1, arr2):
        if(i != i):
            result.append(j)
        else:
            result.append(i)
    return result

tmp = pd.DataFrame(columns=gbs.columns)

j = 0
skip = False

for i in range(len(gbs.Gruppe)-1):
    if(not skip):
        if(gbs.iloc[i]["GBS-CODE"] == gbs.iloc[i+1]["GBS-CODE"]):
            tmp.loc[j] = merge(gbs.iloc[i].values, gbs.iloc[i+1].values)
            j = j+1
            skip = True
        else:
            tmp.loc[j] = gbs.iloc[i].values
            j = j+1
    else:
        skip = False
        
gbs = tmp.copy()
del tmp

In [8]:
gbs['Personen im Haushalt']

0      4
1      1
2      2
3      1
4      1
      ..
574    1
575    1
576    1
577    1
578    1
Name: Personen im Haushalt, Length: 579, dtype: object

### VALUE MAPPING

In [12]:
di1 = {'männlich': 0, "Männlich": 0, 'weiblich': 1, "Weiblich": 1, '[NA] Keine Angabe':np.nan}
gbs.replace({"Geschlecht": di1}, inplace=True)

di2 = {'Hochschulabschluss':5, 
       'Abitur, allgemeine oder fachgebundene Hochschulreife (Gymnasium bzw. EOS)': 4, 
       'Fachhochschulreife (Abschluss einer Fachoberschule)': 4, 
       'Realschulabschluss (Mittlere Reife)': 3,
       'Hauptschulabschluss/Volksschulabschluss': 2,
       'Abschluss Polytechnische Oberschule 10. Klasse (vor 1965: 8. Klasse)': 1, 
       'Anderer Schulabschluss:': 1}
gbs.replace({'Schulabschluss': di2}, inplace=True)

di3 = {
    'Selbstständige(r) Landwirt/-in bzw. Genossenschaftsbauer/-bäuerin':8,
    'Selbstständige (im Handel, im Gastgewerbe, im Handwerk, in der Industrie, der Dienstleistung, auch Ich-AG oder Produktionsgenossenschaft des Handwerks (PGH)-Mitglied)':5,
    'Angestellte(r)':3,
    'Beamter/Beamtin, Richter/-in, Berufssoldat/-in':4,
    'Akademiker/-in in freiem Beruf (Arzt/Ärztin, Rechtsanwalt/-anwältin, Steuerberater/-in u. Ä.)':2,
    '[NA] Keine Angabe':8,
    'keine passende Angabe':8,
    'Ausbildung':8,
    'Arbeiter/-in':3,
    'mithelfende(r) Familienangehörige(r)':1
}
gbs.replace({'Berufsgruppe':di3}, inplace=True)

arbeitszeit = []
for w in gbs.woechentlicheArbeitszeit:
    if w < 20:
        arbeitszeit.append(1)
    elif w >= 20 and w < 30:
        arbeitszeit.append(2)
    elif w >= 30 and w <= 40:
        arbeitszeit.append(3)
    elif w > 40 and w < 50:
        arbeitszeit.append(4)
    else:
        arbeitszeit.append(5)
gbs['woechentlicheArbeitszeit'] = arbeitszeit

di4 = {9:2,
       999:2,
      -3:2,
      -1:2,
      3:2}
allensbach.replace({'Erwerbstätigkeit': di4}, inplace=True)
gbs.replace({'Erwerbstätigkeit':di4}, inplace=True)


gbs.head()

,GBS-CODE,Gruppe,Geschlecht,Alter,Schulabschluss,Berufsgruppe,Erwerbstätigkeit,BRS1,BRS2,BRS3,...,wenig kuenstlerisches Interesse,Gesellig,Andere kritisieren,Gruendlich,Nervoes,Phantasievoll,Druck,Optimismus Zukunft,Zufriedenheit Wahlergebnis,Resilienz
0,GBS+00027,NEGATIV,0.0,48.0,3.0,4.0,1.0,3.0,2.0,3.0,...,trifft ueberhaupt nicht zu (1),trifft voll und ganz zu (5),eher zutreffend (4),eher zutreffend (4),weder noch (3),trifft voll und ganz zu (5),2.0,3.0,2.0,4.0
1,GBS+00032,POSITIV,1.0,68.0,5.0,8.0,1.0,5.0,1.0,5.0,...,trifft ueberhaupt nicht zu (1),eher zutreffend (4),trifft eher nicht zu (2),trifft voll und ganz zu (5),trifft ueberhaupt nicht zu (1),trifft voll und ganz zu (5),4.0,5.0,3.0,1.0
2,GBS+00071,POSITIV,1.0,56.0,3.0,4.0,1.0,4.0,4.0,2.0,...,eher zutreffend (4),trifft eher nicht zu (2),trifft eher nicht zu (2),eher zutreffend (4),weder noch (3),eher zutreffend (4),2.0,3.0,1.0,4.0
3,GBS+00084,POSITIV,1.0,55.0,4.0,3.0,1.0,4.0,2.0,4.0,...,eher zutreffend (4),trifft eher nicht zu (2),weder noch (3),eher zutreffend (4),trifft eher nicht zu (2),weder noch (3),3.0,4.0,4.0,2.0
4,GBS+00092,NaN,1.0,60.0,5.0,5.0,1.0,5.0,2.0,5.0,...,trifft eher nicht zu (2),eher zutreffend (4),trifft eher nicht zu (2),trifft voll und ganz zu (5),trifft eher nicht zu (2),trifft voll und ganz zu (5),4.0,5.0,2.0,2.0


In [13]:
rep1 = {
    '5 Trifft voll und ganz zu':5,
    '4 Eher zutreffend':4,
    '3 Weder noch':3,
    '2 Trifft eher nicht zu':2,
    '1 Trifft überhaupt nicht zu':1,
    
    'trifft voll und ganz zu (5)':5,
    'eher zutreffend (4)':4,
    'weder noch (3)':3,     
    'trifft eher nicht zu (2)':2,
    'trifft ueberhaupt nicht zu (1)':1, 
    
    'Unit nonresponse':np.nan,
    'Not in panel':np.nan,
    'Item nonresponse':np.nan,
    'Not reached':np.nan,
    'Ambiguous answer':np.nan,
    'keine Angabe':np.nan,
     
    -1:np.nan,
    -9:np.nan
}
rep2 = {
     True:1,
     False:0,
    'Ja':1,
    'Nein':0,
    'True':1,
    'False':0,
    
    'Ja, ich würde wählen.':1,
    'Weiß ich nicht':0.5,
    'Ich war nicht wahlberechtigt.':0.5,
    'Ich bin nicht wahlberechtigt.':0.5,
    'Ich war nicht wahlberechtigt':0.5,
    'Ja, ich würde wählen.':1,
    'Nein, ich würde nicht wählen.':0,
    
    'Nie':1,
    'Fast nie':2,
    'Manchmal':3,
    'Meistens':4,
    'Fast Immer':4,
    'Fast immer':4,
    'Immer':4,
    
    'Gar nicht':1,
    'Ein bisschen':2,
    'Einigermaßen':3,
    'Äußerst':4,
    'Erheblich':4
}

gbs = gbs.replace(rep1)
gbs = gbs.replace(rep2)

di3 = {'Türkei':0, 'Österreich': 0, 
       'Item nonresponse': np.nan, 'sonstige, und zwar:': 0, 'Europa':0, 'Deutschland':1, 'Andere':0}
gbs = gbs.replace({'Geburtsland': di3})

di4 = {2.0: 0, 2:0}
gbs = gbs.replace({'Nationalitaet': di4})

di5 = {'Verheiratet/ Eing. LP zus. lebend':1,
       'Verheiratet und lebe mit meinem/r Ehepartner/-in zusammen': 1, 
       'Geschieden/ Eing. LP aufgehoben': 2,
       'Geschieden': 2,
       'Ledig': 2,
       'In eingetragener Lebenspartnerschaft zusammenlebend (gleichgeschlechtlich)':1,
       'Verheiratet und lebe von meinem/meiner Ehepartner/-in getrennt': 0,
       'Eingetragene Lebenspartnerschaft, getrennt lebend (gleichgeschlechtlich)': 0,
       'Eingetragene/r Lebenspartner/-in verstorben (gleichgeschlechtlich)': 0,
       'Verheiratet/ Eing. LP getrennt lebend': 0,
       'Geschieden/ Eing. LP aufgehoben': 1,
       '[NA] Keine Angabe':np.nan,
       'Verwitwet': 1,
       'Verwitwet/ Eing. LP verstorben': 1,
       'Item nonresponse':np.nan}
gbs = gbs.replace({'Familienstand': di5})

di6 = {'Abitur, allgemeine oder fachgebundene Hochschulreife (Gymnasium bzw. EOS)': 3,
       'Abitur, allgemeine oder fachgebundene Hochschulreife': 3,
       'Item nonresponse':np.nan, 
       'Anderer Schulabschluss:':np.nan, 
       'Anderer Schulabschluss':np.nan,
       'Fachhochschulreife, Fachoberschule': 3,
       'Realschulabschluss':2,
       'Polytechnische Oberschule DDR, Abschluss 10. Klasse': 2,
       'Fachhochschulreife (Abschluss einer Fachoberschule)': 2,
       'Schüler/-in': np.nan,
       'Hauptschulabschluss':1,
       'Hochschulabschluss':3,
       'Hauptschulabschluss/Volksschulabschluss': 1,
       'Von der Schule abgegangen ohne Hauptschulabschluss':np.nan,
       'Realschulabschluss (Mittlere Reife)': 2,
       'Polytechnische Oberschule DDR, Abschluss 8.oder 9. Klasse': 2,
       'Abschluss Polytechnische Oberschule 10. Klasse (vor 1965: 8. Klasse)': 2}
gbs = gbs.replace({'Hoechster Bildungsabschluss': di6})

di7 = {1.0: 51,
       2.0: 52,
       3.0:np.nan,
       4.0:np.nan,
      -1.0:np.nan,
      'Geringfügig erwerbstätig, 400-Euro-Job, Minijob':53,
      'Item nonresponse':np.nan,
      'In einer beruflichen Ausbildung/Lehre':50,
      'In Umschulung':54,
      'Altersteilzeit (nicht mehr am Arbeitsplatz)':54,
      '\'Ein-Euro-Job\' (bei Bezug von Arbeitslosengeld II)':52,
      'Gelegentlich oder unregelmäßig beschäftigt':52,
      'Freiwilliger (Wehr-)Dienst':51,
      'Mutterschafts-, Erziehungsurlaub, Elternzeit oder sonstige Beurlaubung':52,
      'Vollzeiterwerbstätig':51,
      'Nicht erwerbstätig':52,
      'Teilzeiterwerbstätig':53}
#df = df.replace({'Erwerbstaetigkeit': di7})

di9 = {'Item nonresponse':np.nan,
        'Missing by filter': np.nan,
       'Angabe verweigert':np.nan,
       'Weiß nicht':np.nan,     
'6000 € und mehr'                     :4,
'5000 bis unter 6000 €'               :4,
'4000 bis unter 5000 €'               :4,
'3200 bis unter 4000 €'               :3,
'2600 bis unter 3200 €'               :2,
'2300 bis unter 2600 €'               :2,
'2000 bis unter 2300 €'               :2,
'1700 bis unter 2000 €'               :1,
'1500 bis unter 1700 €'               :1,
'1300 bis unter 1500 €'               :1,
'1100 bis unter 1300 €'               :1,
'900 bis unter 1100 €'                 :0,
'700 bis unter 900 €'                  :0,
'700 € und weniger'                    :0, 
'4 000\t bis unter\t4 500 Euro'       :4,
'3 750\t bis unter \t4 000 Euro'      :3,
'6 000\t bis unter \t7 500 Euro'      :4,
'3 000\t bis unter \t3 250 Euro'      :3,
'2 000\t bis unter\t2 250 Euro'       :2,
'4 500\t bis unter\t5 000 Euro'       :4,
'5000\t bis unter\t5 500 Euro'        :4,
'2 750\t bis unter \t3 000 Euro'      :2,
'3 250\t bis unter \t3 500 Euro'      :3,
'1 750 \t bis unter\t2 000 Euro'      :1,
'2 250\t bis unter\t2 500 Euro'       :2,
'5 500\t bis unter\t6 000 Euro'       :4,
'500 \t bis unter\t800 Euro'           :0,
'2 500 \t bis unter \t2 750 Euro'     :2,
'7 500 \t bis unter \t10 000 Euro'    :4,
'3 500\t bis unter \t3 750 Euro'      :3,
'1 500\t bis unter\t1 750 Euro'       :1,
'1 250 \t bis unter \t1 500 Euro'     :1,
'801\t bis unter \t1 000 Euro'         :0,
'10 000\t bis unter\t20 000 Euro'     :4,
'1 000 \t bis unter \t1 250 Euro'     :1,
'401 \t bis unter 500 Euro'              :0,
'150 \t bis unter\t400 Euro'             :0,
'unter 150 Euro'                         :0,
'Kein eigenes Einkommen'                 :np.nan,
'kein eigenes Einkommen'                 :np.nan,
'300 bis unter 500 €'                    :0,
'500 bis unter 700 €'                    :0,
'unter 300 €'                            :0,
'5000 € und mehr'                        :4}
gbs = gbs.replace({'Nettoeinkommen Selbst': di9})
gbs = gbs.replace({'Nettoeinkommen Haushalt': di9})

di11 = {
'Beruflich-betriebliche Berufsausbildung':1,
'Universitätsabschluss':2,
'Kein beruflicher Abschluss, nicht beruflicher Ausbildung':3,
'Ausbildung an Fach-, Meister-, Technikerschule,Berufs- oder Fachakademie':4,
'Fachhochschulabschluss':5,
'Beruflich-schulische Ausbildung':6,
'Student/-in':7,
'Noch in beruflicher Ausbildung':8,
'Ausbildung an Fachschule der DDR':9,
'Schüler/-in an berufsorientiertre Aufbau-, Fachschule o. Ä.':10,
'Item nonresponse':np.nan,
'Missing by filter':np.nan,
'Anderer beruflicher Abschluss':11,

'Beruflich-betriebliche Berufsausbildung (Lehre) abgeschlossen':1,
'Universitätsabschluss (z. B. Diplom, Magister, Staatsexamen, Master)':2,
'Fachhochschulabschluss (z. B. Diplom, Master)':5,
'Noch in beruflicher Ausbildung (Berufsvorbereitungsjahr, Auszubildende/r, Praktikant/-in, Student/-in)':8,
'Ausbildung an einer Fach-, Meister-, Technikerschule, Berufs- oder Fachakademie abgeschlossen':4,
'Bachelor an (Fach-)Hochschule abgeschlossen':5,
'Beruflich-schulische Ausbildung (Berufsfachschule, Handelsschule, Vorbereitungsdienst für den mittleren Dienst in der öffentlichen Verwaltung) abgeschlossen':6,
'Promotion':2,
'Anderer Abschluss, und zwar:':np.nan,
'Keinen beruflichen Abschluss und bin nicht in beruflicher Ausbildung':3,
'Ausbildung an einer Fachschule der DDR abgeschlossen':9,
'[NA] Keine Angabe':np.nan   
}
gbs = gbs.replace({'Berufliche Ausbildung': di11})

gbs['Geburtsjahr'] = gbs['Geburtsjahr'].astype(str).str[0:4].astype(int)

absicht = {3:0.5, 2:0, 1:0}
gbs = gbs.replace({'Wahlabsicht': absicht})
absicht2 = {5:1, 4:1}
gbs = gbs.replace({'Wahlabsicht': absicht2})

In [14]:
gbs['Geburtsjahr'] = gbs['Geburtsjahr'].astype(str).str[0:4].astype(int)

In [15]:
repeat_dict = {}
var_list = list(gbs) # list of varnames as strings
for varname in var_list:
        # make a list of all instances of that varname
    test_list = [v for v in var_list if v == varname] 
        # if more than one instance, report duplications in repeat_dict
    if len(test_list) > 1: 
        repeat_dict[varname] = len(test_list)
print(repeat_dict)

{}


In [16]:
allensbach = allensbach.fillna(allensbach.median())
gbs = gbs.fillna(gbs.median())



allensbach['label'] = 0
gbs['label'] = 1

data = pd.concat([gbs, allensbach], sort=True).copy(deep=True)

drop = ['ASKU1', 'ASKU2', 'ASKU3']
data.drop(columns=drop, axis=1, inplace=True)

cols = list(data.columns) 
meta = ['label', 'Id']
for m in meta:
    if m in cols:
        cols.remove(m)

data.head()

,Aktiv,Alles anstrengend,Alter,Andere kritisieren,BRS1,BRS2,BRS3,BRS4,BRS5,BRS6,...,Wahlabsicht,Wahlteilnahme,Zufriedenheit Leben,Zufriedenheit Wahlergebnis,Zurueckhaltend,label,leicht Vertrauen,wenig kuenstlerisches Interesse,woechentlicheArbeitszeit,Ängstlich
0,3.0,2.0,48,4.0,3,2,3,3,3,4,...,1.0,1.0,2.0,2.0,2.0,1,4.0,1.0,4,4.0
1,3.0,2.0,68,2.0,5,1,5,1,5,1,...,1.0,1.0,3.0,3.0,3.0,1,3.0,1.0,1,4.0
2,3.0,2.0,56,2.0,4,4,2,4,2,4,...,1.0,1.0,3.0,1.0,4.0,1,4.0,4.0,3,3.0
3,3.0,2.0,55,3.0,4,2,4,3,4,2,...,1.0,1.0,3.0,4.0,4.0,1,4.0,4.0,3,4.0
4,3.0,2.0,60,2.0,5,2,5,2,5,2,...,1.0,1.0,3.0,2.0,2.0,1,5.0,2.0,3,4.0


In [24]:
data.to_csv(os.path.join(path, 'allensbach_mrs.csv'), sep=',')